In [0]:
!pip install stop-words
!pip install pymorphy2

In [0]:
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from string import punctuation
import pymorphy2
import re
import nltk
import pandas as pd

In [0]:
df = pd.read_csv('/content/drive/My Drive/mailClassifier/mail_parsed_1.csv', encoding = 'utf-8-sig', lineterminator='\n')
df = df[df['content']!='none'].drop('Unnamed: 0',1)
df = df.fillna(' ').reset_index(drop=True)

In [0]:
!nltk.download('punkt')
!nltk.download('stopwords')

Создаем список стоп-слов

In [0]:
stop_words2 = get_stop_words('ru')

stop_words = nltk.corpus.stopwords.words("russian")
stop_words.extend(['что', 'это','наш', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 
                   'к', 'на', '::', '...', 'главная', 'главный', 'сайт', 'страница', 
                   'интернет', 'ооо', 'интернетмагазин','россия', 'украина','год','весь',
                   'который','свой','ваш','мочь','самый','хороший','также','далее','каждый',
                   'последний','являться','очень','ещё','любой','ррсрсррёрррссрё',
                   'пожалуйста','подождать','удалить','игнора','спам'])
stop_words.extend(stop_words2)
stop_words = set(stop_words)

morph = pymorphy2.MorphAnalyzer()

Создаем функцию предобработки текста <br>
Фнукция уберет все символы кроме русских букв, после этого применит лемматизацию к русским словам переводя их в начальную форму, уберет все стоп слова и вернет очищенный текст

In [0]:
def RussianPreprocess(text):
    result = re.sub(r'[^а-яА-ЯёЁ ]', '', text)
    tokens = word_tokenize(result.lower())
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    tokens = [token for token in tokens if token not in stop_words\
              and token != " " \
              and token.strip() not in punctuation]
    text = " ".join(tokens)
    
    return text

Применяем созданную ранее функцию ко всем 3 колонкам нашего датасета, используя мультипроцессинг для ускорения выполнения

In [0]:
content = df['content'].to_list()
titles = df['title'].to_list()
descriptns = df['description'].to_list()

content_length = len(content)

with concurrent.futures.ProcessPoolExecutor() as executor:
    r = list(tqdm.tqdm(executor.map(RussianPreprocess, content), total=content_length))

df['content'] = r

with concurrent.futures.ProcessPoolExecutor() as executor:
    r = list(tqdm.tqdm(executor.map(RussianPreprocess, titles), total=content_length))

df['title'] = r

with concurrent.futures.ProcessPoolExecutor() as executor:
    r = list(tqdm.tqdm(executor.map(RussianPreprocess, descriptns), total=content_length))

df['description'] = r

In [0]:
df.to_csv('/content/drive/My Drive/mailClassifier/mail_lemmatized_1.csv', encoding='utf-8-sig')